91で作成した評価データの各事例に対して，vec(2列目の単語) - vec(1列目の単語) + vec(3列目の単語)を計算し，そのベクトルと類似度が最も高い単語と，その類似度を求めよ．求めた単語と類似度は，各事例の末尾に追記せよ．このプログラムを85で作成した単語ベクトル，90で作成した単語ベクトルに対して適用せよ．

In [1]:
import csv

import numpy as np
import pandas as pd

In [2]:
# 保存時に引数を指定しなかったので'arr_0'に格納されている
matrix_x300 = np.load('./../09.ベクトル空間法 (I)/085.matrix_x300.npz')['arr_0']

print('matrix_x300 Shape:', matrix_x300.shape)

matrix_x300 Shape: (388836, 300)


In [3]:
group_t = pd.read_pickle('./../09.ベクトル空間法 (I)/083_group_t.zip')

In [4]:
# コサイン類似度計算
def get_cos_similarity(v1, v1_norm, v2):
    
    # ベクトルが全てゼロの場合は-1を返す
    if np.count_nonzero(v2) == 0:
        return -1
    else:
        return np.dot(v1, v2) / (v1_norm * np.linalg.norm(v2))

In [5]:
# 類似度が高い単語取得
def get_similar_word(cols):
    
    try:        
        vec = matrix_x300[group_t.index.get_loc(cols[1])] \
              - matrix_x300[group_t.index.get_loc(cols[0])] \
              + matrix_x300[group_t.index.get_loc(cols[2])]
        vec_norm = np.linalg.norm(vec)
        
        # 計算に使った自身の3単語は除外
        cos_sim = [-1 if group_t.index[i] in cols[:3] else get_cos_similarity(vec, vec_norm, matrix_x300[i]) for i in range(len(group_t))]
        index = np.argmax(cos_sim)
        
        cols.extend([group_t.index[index], cos_sim[index]])
        
    except KeyError:
        cols.extend(['', -1])
    return cols

In [6]:
%%time
# 評価データ読み込み
with open('./091.analogy_family.txt') as file_in:
    result = [get_similar_word(line.split()) for line in file_in]

/home/i348221/Apps/python/venv/py36/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


CPU times: user 15min 49s, sys: 10.4 s, total: 15min 59s
Wall time: 16min 41s


In [7]:
with open('092.analogy_word2vec_1.txt', 'w') as file_out:
    writer = csv.writer(file_out, delimiter='\t', lineterminator='\n')
    writer.writerows(result)